In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811633 sha256=2d93184d30e4fd606a6831f1a2afc860aed51749fd3a691539254b0327415795
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd

from surprise import Dataset, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import cross_validate, GridSearchCV

from collections import defaultdict
import statistics
from surprise.model_selection import train_test_split

In [3]:
data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] н
Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
with open("/root/.surprise_data/ml-100k/ml-100k/u.info", "r") as f:
    print(f.read())

943 users
1682 items
100000 ratings



In [5]:
raw_ratings = data.raw_ratings
raw_ratings = [(user_id, item_id, rating) for user_id, item_id, rating, _ in raw_ratings]

columns = ['user_id', 'item_id', 'rating']
df = pd.DataFrame(raw_ratings, columns=columns)

print(df.head())

  user_id item_id  rating
0     196     242     3.0
1     186     302     3.0
2      22     377     1.0
3     244      51     2.0
4     166     346     1.0


In [6]:
mean_rating = df['rating'].mean()
std_rating = df['rating'].std()

print("Mean Rating:", mean_rating)
print("Standard Deviation of Ratings:", std_rating)

Mean Rating: 3.52986
Standard Deviation of Ratings: 1.125673599144316


In [7]:
trainset, testset = train_test_split(data, test_size=0.2)

## SVD

In [8]:
param_grid_svd = {
    'n_epochs': [10, 15, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.04, 0.06],
    'n_factors': [1, 20, 50],
    'init_mean': [mean_rating],
    'init_std_dev': [0.1, 1]
}

In [9]:
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5, n_jobs=-1, joblib_verbose=5)

grid_search_svd.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  4.8min finished


In [31]:
print(grid_search_svd.best_params)
print(grid_search_svd.best_score['rmse'])
print(grid_search_svd.best_score['mae'])
best_params_svd = grid_search_svd.best_params['rmse']
print(best_params_svd)

{'rmse': {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}, 'mae': {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}}
0.9517164111984486
0.7498014386058419
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}


In [11]:
algo_svd = SVD(n_epochs=20, n_factors=1, lr_all=0.005, reg_all=0.06, init_mean=3.52986, init_std_dev=0.1)

algo_svd.fit(trainset)

predictions = algo_svd.test(testset)

rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.9493
MAE:  0.7464


## SVD++

In [12]:
param_grid_svd_plus = {
    'n_epochs': [10, 15, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.04, 0.06],
    'n_factors': [1, 20, 50],
    'init_mean': [mean_rating],
    'init_std_dev': [0.1]
}

In [13]:
grid_search_svd_plus = GridSearchCV(SVDpp, param_grid_svd_plus, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=5)

grid_search_svd_plus.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 35.7min finished


In [30]:
print(grid_search_svd_plus.best_params)
print(grid_search_svd_plus.best_score['rmse'])
print(grid_search_svd_plus.best_score['mae'])
best_params_svd_plus = grid_search_svd_plus.best_params['rmse']
print(best_params_svd_plus)

{'rmse': {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}, 'mae': {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}}
1.7480287618654504
1.401785630750639
{'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}


In [15]:
algo_svd_plus = SVDpp(n_epochs=20, n_factors=1, lr_all=0.002, reg_all=0.04, init_mean=3.52986, init_std_dev=0.1)

algo_svd_plus.fit(trainset)

predictions = algo_svd_plus.test(testset)

rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.6651
MAE:  1.3385


## NMF

In [16]:
param_grid_nmf = {
    'n_epochs': [10, 15, 20],
    'n_factors': [1, 20, 50],
    'reg_pu': [0.02, 0.04, 0.06],
    'reg_qi': [0.02, 0.04, 0.06]
}

In [17]:
grid_search_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=3, n_jobs=-1, joblib_verbose=5)

grid_search_nmf.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   50.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  3.7min finished


In [32]:
print(grid_search_nmf.best_params)
print(grid_search_nmf.best_score['rmse'])
print(grid_search_nmf.best_score['mae'])
best_params_nmf = grid_search_nmf.best_params['rmse']
print(best_params_nmf)

{'rmse': {'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}, 'mae': {'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}}
1.1046978665160914
0.8408488437532761
{'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}


In [19]:
algo_nmf = NMF(n_epochs=20, n_factors=20, reg_pu=0.06, reg_qi=0.06)

algo_nmf.fit(trainset)

predictions = algo_nmf.test(testset)

rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.1071
MAE:  0.8426


In [34]:
algos = [SVD(**best_params_svd), SVDpp(**best_params_svd_plus), NMF(**best_params_nmf)]
for algo in algos:
    results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    print(f"Algorithm: {algo.__class__.__name__}")
    print(f"RMSE: {sum(results['test_rmse']) / len(results['test_rmse'])}")
    print(f"MAE: {sum(results['test_mae']) / len(results['test_mae'])}")
    print("---")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9575  0.9542  0.9517  0.9408  0.9504  0.9509  0.0056  
MAE (testset)     0.7529  0.7532  0.7507  0.7416  0.7500  0.7497  0.0042  
Fit time          0.86    0.88    0.92    0.88    0.91    0.89    0.02    
Test time         0.45    0.15    0.17    0.33    0.15    0.25    0.12    
Algorithm: SVD
RMSE: 0.9509002571999007
MAE: 0.7496547858309863
---
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8596  1.9716  1.9984  1.9722  1.9771  1.9558  0.0491  
MAE (testset)     1.5312  1.6365  1.6652  1.6423  1.6428  1.6236  0.0472  
Fit time          8.07    8.10    8.09    8.26    7.73    8.05    0.18    
Test time         5.17    5.24    5.12    5.49    6.23    5.45    0.41    
Algorithm: SVDpp
RMSE: 1.9557672665133652
MAE: 1.623592591226176
---
Eva

# Висновок

1. SVD:
- Найкращі параметри: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.06, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}
- Найкраща RMSE: 0.9517
- Найкраща MAE: 0.7498
- Середні часи навчання (Fit time) та часи тестування (Test time) на кожному спліті заняли відповідно приблизно 0.89 та 0.25 секунди.

2. SVD++:
- Найкращі параметри: {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04, 'n_factors': 1, 'init_mean': 3.52986, 'init_std_dev': 0.1}
- Найкраща RMSE: 1.7480
- Найкраща MAE: 1.4018
- Середні часи навчання (Fit time) та часи тестування (Test time) на кожному спліті заняли відповідно приблизно 8.05 та 5.45 секунд.

3. NMF:
- Найкращі параметри: {'n_epochs': 20, 'n_factors': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}
- Найкраща RMSE: 1.1047
- Найкраща MAE: 0.8408
- Середні часи навчання (Fit time) та часи тестування (Test time) на кожному спліті заняли відповідно приблизно 1.35 та 0.23 секунди.


SVD показує найкращі результати серед усіх трьох алгоритмів з найнижчими значеннями RMSE (0.9517) та MAE (0.7498).

SVD++ показує значно гірші результати порівняно з SVD, з набагато вищими значеннями RMSE (1.7480) та MAE (1.4018). SVD++ враховує неявний відгук, крім явних оцінок, але в даному випадку це не призвело до поліпшення результатів. SVDpp є найповільнішим алгоритмом, так як виконується тривало через високий обчислювальний ресурс, який потрібен для обробки більш складних моделей.

NMF показує кращі результати, ніж SVD++, але гірші, ніж SVD. Його RMSE (1.1047) та MAE (0.8408) нижчі, ніж у SVD++, але все ж вищі, ніж у SVD. NMF має інший підхід до факторизації матриць, який спрямований на розкладання матриць на невід'ємні компоненти.

Загалом, SVD виділяється як найефективніший алгоритм для даного набору даних, надаючи найкращу точність прогнозування для рейтингів фільмів.

